In [ ]:
import pandas as pd
import add_isocountry
from importlib import reload
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pio.renderers.default = "notebook"

In [ ]:
reload(add_isocountry)
df = pd.read_csv("../../web/data/pcc_energy_joined.csv", index_col=0)
df = add_isocountry.add_isocountry_column(df, "Entity")

Noticed that the continents and the world is included as entity. Although Interesting does a disservice to the current dataset so they are removed

In [ ]:
df = df.drop(
    df[
        df.Entity.isin(
            ["Africa", "Europe", "World", "Asia", "North America", "South America"]
        )
    ].index
)

In [ ]:
df.to_csv("../../web/data/pcc_energy_joined_country_codes.csv")

In [ ]:
group_df = df.groupby("Year").sum().unstack().reset_index()
group_df.columns = ["x", "Year", "y"]

In [ ]:
fig = make_subplots(
    rows=1,
    cols=2,
    column_widths=[4, 1],
    specs=[[{"type": "choropleth"}, {"type": "xy"}]],
    subplot_titles=(
        "world map of energy_consumption",
        "Bar chart of the distribution of energy sources <br>Per capita (kwh)",
    ),
)
world_slice = df[df.Year == 1965]
bar_slice = group_df[group_df.Year == 1965]

fig.add_trace(
    go.Choropleth(
        locations=world_slice["country_code"],
        z=world_slice["Fossil Fuels per capita (kWh)"],
        colorscale="Reds",
        colorbar_ticksuffix=" kWh",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Bar(
        x=bar_slice.y,
        y=[x.replace("per capita (kWh)", "") for x in bar_slice.x],
        marker=dict(color="forestgreen"),
        orientation="h",
    )
)

fig.update_geos(
    projection_type="natural earth2",
    landcolor="white",
    oceancolor="#2999BC",
    showocean=True,
)

fig.update_layout(template="gridon")

fig.data[0].colorbar.x = -0.1

In [ ]:
years = df.Year.unique()
frames = [
    dict(
        name=str(year),
        data=[
            go.Choropleth(
                locations=df[df.Year == year]["country_code"],
                z=df[df.Year == year]["Fossil Fuels per capita (kWh)"],
                colorscale="Reds",
                colorbar_ticksuffix=" kWh",
            ),
            go.Bar(
                x=group_df[group_df.Year == year]["y"],
                y=[
                    x.replace("per capita (kWh", "")
                    for x in group_df[group_df.Year == year].x
                ],
                orientation="h",
            ),
        ],
        traces=[0, 1],
    )
    for year in years
]

In [ ]:
update_menus = [
    dict(
        type="buttons",
        buttons=[dict(
            label="Play",
            method="animate",
            args=[
                [f"{year}" for year in years],
                dict(
                    frame=dict(duration=500, redraw=True),
                    transition=dict(
                        duration=0,
                    ),
                    easing="linear",
                    fromcurrent=True,
                    mode="immediate",
                ),
            ],
        )],
        direction="left",
        pad=dict(
            r=10,
            t=85
        ),
        showactive=True,
        x = .1,
        y = 0,
        xanchor="right",
        yanchor="top"
    ),
]
sliders = [{
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {"font": {"size": 16}, "prefix": "Year: ", "visible": True, "xanchor": "right"},
    "transition": {"duration": 500.0, "easing": "linear"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": [{
        "args": [[year], {"frame": {"duration": 500, "easing": "linear", "redraw": False},
                       "transition": {"duration": 0, "easing": "linear"}}],
        "label": str(year), "method": "animate"} for year in years]
}]

In [ ]:
fig.update(frames=frames)
fig.update_layout(updatemenus=update_menus, sliders=sliders)

In [ ]:
col_names = set(group_df.columns) - set(["Year"])

In [ ]:
fig = px.bar(
    group_df,
    x="y",
    y="x",
    animation_frame="Year",
    orientation="h",
)
fig.show()

In [ ]:
group_df.max()

Seeing format for px.choropleth

In [ ]:
import plotly.express as px

In [ ]:
df = px.data.gapminder().query("year==2007")

In [ ]:
df.head()